Практическая работа №6

Создание двух разных моделей

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель номер 1 будет простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
model1.fit(train_images, train_labels, epochs=5)
model1.save('mnist_model_1.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8746 - loss: 0.4354
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9649 - loss: 0.1229
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9763 - loss: 0.0783
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9819 - loss: 0.0602
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9859 - loss: 0.0449


In [2]:
# Модель 2 будет свёрточная нейронная сеть
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
model2.save('mnist_model_2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 48s 25ms/step - accuracy: 0.9103 - loss: 0.2979
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 24ms/step - accuracy: 0.9836 - loss: 0.0520
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 23ms/step - accuracy: 0.9903 - loss: 0.0300
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 84s 24ms/step - accuracy: 0.9938 - loss: 0.0197
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - accuracy: 0.9957 - loss: 0.0133


Атака полносвязной нейронной сети

In [6]:
def fgsm_attack(image, epsilon, gradient):
  changed_image = image + epsilon * np.sign(gradient)
  changed_image = np.clip(changed_image, 0, 1)
  return changed_image


def generate_adversarial_example(model, image, label, epsilon):
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))
    adversarial_images = np.array(adversarial_images)
    print("Shape of adversarial_images:", adversarial_images.shape)
    return adversarial_images

# Генерация атакованных изображений
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)

Shape of adversarial_images: (10000, 28, 28)


Оценка противоречивых примеров на обеих моделях

In [7]:
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Точность первой модели на атакованных изображениях: {acc1}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0826 - loss: 6.6944
Точность первой модели на атакованных изображениях: 0.10779999941587448


In [8]:
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Точности второй модели на атакованных изображениях модели 1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9578 - loss: 0.1458
Точности второй модели на атакованных изображениях модели 1: 0.9635999798774719


Анализ переносимости атак

In [9]:
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Точность первой модели на основе атакованных изображений модели 2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9532 - loss: 0.1552
Точность первой модели на основе атакованных изображений модели 2: 0.9563999772071838


В ходе выполнения работы было исследовано воздействие атаки по переносу, при которой противоречивые примеры, сгенерированные для одной модели, использовались для атаки на другую. Для реализации атаки использовался метод FGSM

В результате было выявлено, что модель 1, на которой были сгенерированы противоречивые примеры с использованием FGSM, продемонстрировала ощутимое падение точности с 98% до 11%, что свидетельствует о её высокой уязвимости к данной атаке. Модель 2 показала гораздо меньший эффект от атаки — её точность снизилась лишь на 3%, что указывает на большую устойчивость к переносу атак, созданных для другой модели.

Дополнительно были сгенерированы противоречивые примеры с использованием FGSM для модели 2, и они были протестированы на модели 1. В этом случае точность модели 1 также снизилась, но падение составляет около 4%.

Атака по переносу с использованием FGSM может существенно ослабить точность модели, особенно если примеры сгенерированы для неё самой. Другая модель, показавшая более высокую устойчивость, остаётся менее подверженной таким атакам.